# IMPORTANT

This notebook has 2 parts to it, one of which is checking for necessary residues between the original protein and the new one, and the second part which is running autodock Vina for checking affinity of the new protein.

For PART 1, you need:
- The pdb of the original protein
- The pdb of the newly generated protein
- The contigs of the newly generated protein that were chosen (insert below in Aligning Proteins section)

Note: If you are doing something other than beta lactamase, you would want to change the orig_aligns variable as it check for necessary residues in beta lactamase, not other proteins.

For PART 2, you need:
- The pdb of newly generated protein
- The sdf (ligand file) of the protein

To upload these files, go to the left bar and click on the folder. Click the upload file button and upload the necessary files

NOTE: After running this cell below, your colab WILL crash. This is normal, just continue running the rest of the cells as usual

In [ ]:
# @title (1) Install Condacolab (< 1min)
%%time

! pip install -q condacolab
import condacolab
condacolab.install()

#@markdown > An automatic restart of the kernel is expected after the execution of this block.
#@markdown >
#@markdown > Stay connected to the same runtime and proceed to the next code block!


# File Names

Input the names of all of the files you plan on using here and make sure to upload the file using the instructions above, if you're not planning on uploading all of them, leave the field blank. **ADD THE .pdb or .sdf part at the end**

In [ ]:
#@markdown # Part 1
orig_protein = "1btl.pdb" #@param {type:"string"}
#@markdown Insert original protein pdb file here
new_protein = "best_design0.pdb" #@param {type:"string"}
#@markdown Insert novel protein pdb file here
#[70, 73, 130, 132, 166, 234, 237]
orig_aligns =  [70, 73, 130, 132, 166, 234, 237] #@param
#@markdown - Change this orig_aligns if you want to compare different residues than the ones given here (make sure to put it in list format like shown)
# "4-4/A68-84/26-26/A125-238/14-14"
#"A0-49/3-9/A56-288"
#"0-68/A69-276"
resid_format = "3-3/A68-84/10-10/A125-190/50-50/A230-238/5-5" #@param {type: "string"}
#@markdown - This is where you put the contigs of your new protein (This should be inside the csv file)

#@markdown ---
#@markdown # Part 2
sdf_protein = "PNN_ideal.sdf" #@param {type:"string"}
#@markdown - Insert sdf of original protein here


In [ ]:
'''
def find_alignment(contigs, alignment_inds):
    locations = alignment_inds.copy()
    new_locs = []
    contigs = contigs.split("/")
    A_region = [i[1:].split("-") for i in contigs if "A" in i]
    added_region = [int(i.split("-")[0]) for i in contigs if "A" not in i]

    while locations:
        for i in range(len(A_region)):
            region = A_region[i]
            if int(region[0]) <= locations[0] < int(region[1]):  # Check if the location falls in the current region
                loc = (locations[0] - int(region[0]) +
                       sum(added_region[0:i+1]) +  # Sum of added regions before this one
                       sum([int(x[1]) - int(x[0]) for x in A_region[:i]]) +  # Sum of A region lengths before this one
                       i + 1)  # Add one for each inserted region
                # print(locations[0])
                # print(region[0])
                # print(sum(added_region[0:i+1]))
                # print(sum([int(x[1]) - int(x[0]) for x in A_region[:i]]))
                # print(i)
                new_locs.append(loc)
                locations.pop(0)  # Remove the processed location
                break  # Stop processing this location
    return new_locs

new_aligns = find_alignment(resid_format, orig_aligns)
new_aligns
'''

# Part 1

In [ ]:
!conda install -c conda-forge -c schrodinger pymol-bundle -q

Do not rerun this line without disconnecting and deleting runtime or running cell below, else it will put the images over each other and keep appending more to the txt's. This line takes a while to run

Most relevant atoms to search are:
OG for Resid 70
NZ for Resid 73
OG for Resid 130
OD1 for Resid 132
OE1/OE2 for Resid 166
NZ for Resid 234
N for Resid 237

In [ ]:
import pymol
from pymol import cmd
new_aligns = []
cmd.delete("all")
cmd.load(f"{orig_protein}", "target")  # Target structure
cmd.load(f"{new_protein}", "mobile")  # Mobile structure

base_atoms = ['OG', 'NZ', 'OG', 'OD1', 'OE1', 'NZ', 'N']

# Check if both objects are loaded
print("Target Atom Count:", cmd.count_atoms("target"))
print("Mobile Atom Count:", cmd.count_atoms("mobile"))

# Align mobile structure onto the target
cmd.align("target", "mobile")
i = 0

for orig_res in orig_aligns:
    #cmd.select("orig_residue", f"target and resi {orig_res}")

    orig_ca = cmd.get_model(f"target and resi {orig_res} and name {base_atoms[i]}").atom[0].coord

    min_dist = float("inf")
    best_match = None

    for new_res in range(1, cmd.count_atoms("mobile")):
        try:
            new_ca = cmd.get_model(f"mobile and resi {new_res} and name {base_atoms[i]}").atom[0].coord
            dist = ((orig_ca[0] - new_ca[0])**2 +
                    (orig_ca[1] - new_ca[1])**2 +
                    (orig_ca[2] - new_ca[2])**2)**0.5  # Euclidean distance

            if dist < min_dist:
                min_dist = dist
                best_match = new_res
        except IndexError:
            continue  # Skip residues without a CA atom
    new_aligns.append(best_match)
    i+=1
    print(f"Residue {orig_res} in target maps to {best_match} in mobile with {min_dist:.2f} Å")



In [ ]:
#!rm -rf Residues/ #ONLY uncomment this if you want to rerun the line below again and you didn't disconnect and delete runtime

In [ ]:
import pymol
from pymol import cmd
import os

# Initialize PyMOL
cmd.reinitialize()

# Load the target and mobile structures
cmd.load(f"{orig_protein}", "target")  # Target structure
cmd.load(f"{new_protein}", "mobile")  # Mobile structure

# Check if both objects are loaded
print("Target Atom Count:", cmd.count_atoms("target"))
print("Mobile Atom Count:", cmd.count_atoms("mobile"))

# Align mobile structure onto the target
cmd.align("target", "mobile")

# Select Serine 130
#130
#cmd.select("within_6A", "Ser130_target around 6")

# Select corresponding Serine in mobile
#cmd.select("Ser53_mobile", "resi 157 and mobile") #53

# Measure distances between the selected groups
#cmd.distance("130_gap", "mobile_within_6A", "within_6A")
#print(cmd.get_model("Ser53_mobile").atom)

#cmd.zoom()
#cmd.png(f'test.png', 500, 500, dpi = 300, ray = 1)
os.system('mkdir /content/Residues')

t = 0
for i in range(len(orig_aligns)):
  cmd.select(f"Ser{orig_aligns[i]}_target", f"resi {orig_aligns[i]} and target")
  cmd.select(f"Ser{new_aligns[i]}_mobile", f"resi {new_aligns[i]} and mobile")

  os.system(f'mkdir /content/Residues/Orig_{orig_aligns[i]}')
  print(f'For Original Residue {orig_aligns[i]} and New Residue {new_aligns[i]}')
  file_path = f"/content/Residues/Orig_{orig_aligns[i]}/distances.txt"

  orig_res = f"target and resi {orig_aligns[i]}"
  new_res = f"mobile and resi {new_aligns[i]}"
  distance = cmd.distance("dist", f"{orig_res}", f"{new_res}")
  with open(file_path, "a") as f:
    f.write(f"Distance between alpha carbons at residue {orig_aligns[i]} in original and residue {new_aligns[i]} in novel protein: {distance} Å\n\n")

  for atom1 in cmd.get_model(f"Ser{orig_aligns[i]}_target").atom:
      for atom2 in cmd.get_model(f"Ser{new_aligns[i]}_mobile").atom:

        if atom1.name == atom2.name:
          atom1_selection = f"target and resi {orig_aligns[i]} and name {atom1.name}" #130
          atom2_selection = f"mobile and resi {new_aligns[i]} and name {atom2.name}" #53

          cmd.delete("focus_atoms")
          cmd.delete("distance_lines")
          cmd.delete("dist*")  # Remove all previous distance measurements
          #cmd.hide("everything")

          #dist = cmd.distance(f"dist_{atom1.index}_{atom2.index}", f"Ser130_target and id {atom1.index}", f"Ser53_mobile and id {atom2.index}")
          #cmd.select("focus_atoms", f"(id {atom1.index} and Ser130_target) or (id {atom2.index} and Ser53_mobile)")
          dist = cmd.distance("distance_lines", atom1_selection, atom2_selection)

          cmd.hide("labels", "dist*")
          cmd.hide("lines", "dist*")

          cmd.select("focus_atoms", f"({atom1_selection}) or ({atom2_selection})")
          cmd.zoom("focus_atoms", buffer = 5)  # Zoom in on the selected atoms

          # Represent the atoms and distance clearly
          cmd.show("sticks", "focus_atoms")
          cmd.label("focus_atoms", "name")
          cmd.show("spheres", "focus_atoms")
          cmd.set("sphere_scale", 0.3)
          cmd.color("yellow", f"id {atom1_selection}")  # Highlight atom in target
          cmd.color("cyan", f"id {atom2_selection}")
          cmd.color("magenta", "distance_lines")  # Highlight bond in magenta

          cmd.rotate("x", 30)  # Rotate around the x-axis by 30 degrees
          cmd.rotate("y", 20)  # Rotate around the y-axis by 20 degrees
          cmd.rotate("z", 45)
          cmd.translate([1.7, 0, 0], "focus_atoms")

          # Save a high-quality image of the view
          cmd.png(f"/content/Residues/Orig_{orig_aligns[i]}/distance_{atom1.name}_{atom2.name}.png", width=1000, height=800, dpi=300, ray=1)
          with open(file_path, "a") as f:
            f.write(f"Distance between {atom1.name} in original and {atom2.name} in new is {dist}\n")
          #print(f"Distance between {atom1.name} in original and {atom2.name} in new is {dist}")
          t+=1
  #print()


# Optional: Save session for further inspection
#cmd.save("session.pse")


# Part 2

In [ ]:
# @title (2) Install Packages and Data (~ 5min)
%%time

# Install Reduce2 (cctbx-base)
! conda install cctbx-base

!conda install openbabel


# Install Prody and py3Dmol
! pip install prody py3Dmol


# Download Phenix Project geostd (restraint) Library
goestd_repo = "https://github.com/phenix-project/geostd.git"
! git clone {goestd_repo}


# Install Meeko (develop branch) and Dependencies
! conda install numpy scipy rdkit
! git clone --single-branch --branch develop https://github.com/forlilab/Meeko.git
! cd Meeko; pip install --use-pep517 -e .; cd ..


# Install Scrubber (develop branch)
! git clone --single-branch --branch develop https://github.com/forlilab/scrubber.git
! cd scrubber; pip install --use-pep517 -e .; cd ..


# Download Vina Executables
! wget https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_1.2.5_linux_x86_64
! mv vina_1.2.5_linux_x86_64 vina; chmod +x vina;

#@markdown > List of Python packages installed:
#@markdown > - (conda) cctbx-base numpy scipy rdkit
#@markdown > - (pip) prody py3Dmol
#@markdown > - (pip) (develop branch on GitHub) meeko scrubber

#@markdown > Executable and data downloaded:
#@markdown > - (release on GitHub) ccsb-scripps/AutoDock-Vina: vina_1.2.5_linux_x86_64
#@markdown > - (repository on GitHub) Phenix-project/geostd

In [ ]:
# @title (2) Install Packages and Data (~ 2min)
%%time

# Get environment configuration from Git Repository
setup_repo="https://github.com/rwxayheee/colab_setup"
!git clone {setup_repo}

# Run setup script
!chmod +x colab_setup/basic_setup.sh
!bash colab_setup/basic_setup.sh

In [ ]:
# @title (3) Import Modules & Locate Command Line Scripts (< 1s)
%%time

# Import modules
import sys, platform
from prody import *
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import AllChem
import rdkit, py3Dmol
print("rdkit version:", rdkit.__version__)
print("py3Dmol version:", py3Dmol.__version__)
from ipywidgets import interact, IntSlider
import ipywidgets, copy
from IPython.display import display, Markdown


# Helper functions
def locate_file(from_path = None, query_path = None, query_name = "query file"):

    if not from_path or not query_path:
        raise ValueError("Must specify from_path and query_path")

    possible_path = list(from_path.glob(query_path))

    if not possible_path:
        raise FileNotFoundError(f"Cannot find {query_name} from {from_path} by {query_path}")

    return_which = (
        f"using {query_name} at:\n"
        f"{possible_path[0]}\n"
    )
    print(return_which)

    return possible_path[0]


# Commandline scripts
scrub = locate_file(from_path = Path("/usr/local/bin"), query_path = "scrub.py", query_name = "scrub.py")
mk_prepare_ligand = locate_file(from_path = Path("/usr/local/bin"), query_path = "mk_prepare_ligand.py", query_name = "mk_prepare_ligand.py")
mk_prepare_receptor = locate_file(from_path = Path("/usr/local/bin"), query_path = "mk_prepare_receptor.py", query_name = "mk_prepare_receptor.py")
mk_export = locate_file(from_path = Path("/usr/local/bin"), query_path = "mk_export.py", query_name = "mk_export.py")


# Locate reduce2 in conda install prefix
full_py_version = platform.python_version()
major_and_minor = ".".join(full_py_version.split(".")[:2])
env_path = Path("/usr/local") # default conda install prefix on Colab
reduce2_path = f"lib/python{major_and_minor}/site-packages/mmtbx/command_line/reduce2.py"
reduce2 = locate_file(from_path = env_path, query_path = reduce2_path, query_name = "reduce2.py")


# Locate geostd in current path
geostd_path = locate_file(from_path = Path.cwd(), query_path = "geostd", query_name = "geostd")

#@markdown > Version of imported modules and the location of command line scripts will be reported to output.
#@markdown >
#@markdown > Make sure there are no errors and proceed to the next code block!


In [ ]:
!conda install openbabel

In [ ]:
'''
!obabel PNM_ideal.sdf -O PNM_ideal.pdbqt
!obabel best_design0.pdb -O best_design0.pdbqt -xr
!obabel PNM_ideal.sdf -O PNM_ideal.pdb
'''
!obabel {sdf_protein} -O {sdf_protein.split('.')[0]}.pdbqt
!obabel {new_protein} -O {new_protein.split('.')[0]}.pdbqt -xr
!obabel {sdf_protein} -O {sdf_protein.split('.')[0]}.pdb


# Docking Calculation

In these steps, the exmaple docking calculation is demonstrated in a customizable setup.

*Major customizable variables*

- Ligand Smiles string: `ligand_Smiles`
- Designated pH for ligand preparation: `pH`
- Receptor PDB ID: `pdb_token`
- ProDy selection for receptor atoms: `receptor_selection`
- ProDy selection for box center calculation: `ligand_selection`
- Size of box: `size_x`, `size_y`, `size_z`
- Exhaustiveness of docking: `exhaustiveness`

In [ ]:
'''
!git clone https://github.com/sarisabban/Rg.git
val = !python3 Rg/Rg.py best_design0.pdb
val = val.n
val = float(val.split('=')[1].strip())*2.857
val
'''

In [ ]:
from Bio.PDB import PDBParser
import numpy as np

def calculate_radius_of_gyration_biopython(pdb_file):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file)

    atoms = list(structure.get_atoms())
    masses = []
    positions = []

    # Dictionary of approximate atomic masses (in atomic mass units)
    atomic_masses = {
        'H': 1.008, 'C': 12.011, 'N': 14.007, 'O': 15.999, 'P': 30.974, 'S': 32.06
    }

    # Extract masses and positions
    for atom in atoms:
        element = atom.element
        if element in atomic_masses:
            masses.append(atomic_masses[element])
            positions.append(atom.coord)

    masses = np.array(masses)
    positions = np.array(positions)

    # Total mass of the protein
    total_mass = np.sum(masses)

    # Calculate the center of mass
    center_of_mass = np.sum(masses[:, np.newaxis] * positions, axis=0) / total_mass

    # Calculate squared distances of each atom from the center of mass
    squared_distances = np.sum((positions - center_of_mass)**2, axis=1)

    # Calculate the radius of gyration
    radius_of_gyration = np.sqrt(np.sum(masses * squared_distances) / total_mass)

    return radius_of_gyration

# Example usage
pdb_file = new_protein #PNM_ideal.pdb"  # Replace with your file path (works with PDBQT too)
val = calculate_radius_of_gyration_biopython(pdb_file)*2.857
print(f"Radius of gyration: {val:.3f} Å")


In [ ]:
def calculate_center(pdbqt_file):
    x_sum = y_sum = z_sum = 0.0
    atom_count = 0

    with open(pdbqt_file, 'r') as file:
        for line in file:
            if line.startswith("ATOM") or line.startswith("HETATM"):
                x = float(line[30:38].strip())
                y = float(line[38:46].strip())
                z = float(line[46:54].strip())
                x_sum += x
                y_sum += y
                z_sum += z
                atom_count += 1

    center_x = x_sum / atom_count
    center_y = y_sum / atom_count
    center_z = z_sum / atom_count
    return center_x, center_y, center_z

# Example usage
protein_center = calculate_center(f"{new_protein.split('.')[0]}.pdbqt")
print(f"Center Coordinates: {protein_center}")


In [ ]:
file = 'config.txt'
with open(file, "w") as file:
  file.write(f"""center_x = {protein_center[0]}
center_y = {protein_center[1]}
center_z = {protein_center[2]}
size_x = {val}
size_y = {val}
size_z = {val}""")

In [ ]:
# @title # 2. Docking with Vina Scoring Function (~ 3min)
%%time
#@markdown In this step, the docking calculation is executed by ***Vina***.

#@markdown > Specify the names of inputs. You may include any additional options in `configTXT`.
receptorPDBQT = f"{new_protein.split('.')[0]}.pdbqt" #@param {type:"string"}
ligandPDBQT = f"{sdf_protein.split('.')[0]}.pdbqt" #@param {type:"string"}
configTXT = "config.txt" #@param {type:"string"}
#@markdown > Specify the exhaustiveness of docking. In this example, we use a relatively low `exhaustiveness = 8` for demonstration purposes.
exhaustiveness = 8 #@param {type:"raw"}
#@markdown > A name for the docking output PDBQT file is required.
outputPDBQT = "output.pdbqt" #@param {type:"string"}
outputTXT = "affinity_results.txt"

! ./vina --receptor {receptorPDBQT} --ligand {ligandPDBQT} --config {configTXT} \
       --exhaustiveness {exhaustiveness} \
       --out {outputPDBQT} | tee {outputTXT}

In [ ]:
# @title # 3. Export and Visualize Docked Poses (~ 1s)
%%time
import prody as pr
#@markdown In this step, the docking output is converted to atomistic SDF by **mk_export.py**.

# Export Docked Poses
#@markdown > A name for the result SDF file is required.
#dock_outSDF = "1iep_STI_vina_out.sdf" #@param {type:"string"}
#! python {mk_export} {outputPDBQT} -s {dock_outSDF}

#@markdown > Finally (and optionally) for visualization:
#@markdown >
#@markdown > The ***Py3DMol*** view will include object from the following files and specs:
# Previously Generated Receptor Files
#receptorPDB = "1iep_receptorFH.pdb" #@param {type:"string"}
receptorPDB = new_protein
prody_ligandPDB = sdf_protein.split('.')[0] + ".pdb" #'PNM_ideal.pdb'
#boxPDB = "1iep_receptorFH.box.pdb" #@param {type:"string"}

def find_hydrogen_bonds(receptorPDB, ligPDB, max_distance=3.5):
    """Find hydrogen bonds based on distance criteria."""
    receptor = pr.parsePDB(receptorPDB)
    ligand = pr.parsePDB(ligPDB)

    hbonds = []

    for atom1 in receptor.iterAtoms():
        if atom1.getElement() in ['O', 'N']:  # Potential acceptors/donors
            for atom2 in ligand.iterAtoms():
                if atom2.getElement() in ['O', 'N']:  # Potential acceptors/donors
                    dist = pr.calcDistance(atom1, atom2)
                    if dist <= max_distance:  # Typical H-bond distance
                        hbonds.append((atom1.getCoords(), atom2.getCoords()))

    return hbonds


#refligPDB = 'LIG.pdb' #@param {type:"string"}
#reflig_resn = 'STI' #@param {type:"string"}

def Receptor3DView(receptorPDB = None, boxPDB = None, ligPDB = None):

    view = py3Dmol.view()
    view.setBackgroundColor('white')

    #view.addModel(open(boxPDB, 'r').read(),'pdb')
    #view.addStyle({'stick': {}})
    #view.zoomTo()

    view.addModel(open(receptorPDB, 'r').read(),'pdb')
    view.addStyle({'cartoon': {'color':'spectrum', 'opacity': 0.5}})

    if ligPDB is not None:
      view.addModel(open(ligPDB, 'r').read(), 'pdb')
      view.addStyle({'hetflag': True}, {'stick': {}})

    hbonds = find_hydrogen_bonds(receptorPDB, ligPDB)

    # Add hydrogen bonds to visualization
    for donor, acceptor in hbonds:
        view.addCylinder({
            'start': {'x': donor[0], 'y': donor[1], 'z': donor[2]},
            'end': {'x': acceptor[0], 'y': acceptor[1], 'z': acceptor[2]},
            'color': 'yellow',
            'radius': 0.1,
            'dashed': True
        })

    return view

Receptor3DView(receptorPDB = receptorPDB, \
               boxPDB = None, \
               ligPDB = prody_ligandPDB).show()

In [ ]:
import py3Dmol
import prody as pr

def find_hbond_residues(receptorPDB, ligPDB, max_distance=3.5):
    """Identify receptor residues involved in hydrogen bonding with the ligand."""
    receptor = pr.parsePDB(receptorPDB)
    ligand = pr.parsePDB(ligPDB)

    hbond_residues = set()

    for atom1 in receptor.iterAtoms():
        if atom1.getElement() in ['O', 'N']:  # Potential acceptors/donors
            for atom2 in ligand.iterAtoms():
                if atom2.getElement() in ['O', 'N']:  # Potential acceptors/donors
                    dist = pr.calcDistance(atom1, atom2)
                    if dist <= max_distance:  # Typical H-bond distance
                        hbond_residues.add(atom1.getResnum())  # Store residue number

    return hbond_residues

def find_hydrogen_bonds(receptorPDB, ligPDB, max_distance=3.5):
    """Find hydrogen bonds based on atom distances and return coordinate pairs."""
    receptor = pr.parsePDB(receptorPDB)
    ligand = pr.parsePDB(ligPDB)

    hbonds = []

    for atom1 in receptor.iterAtoms():
        if atom1.getElement() in ['O', 'N']:  # Potential acceptors/donors
            for atom2 in ligand.iterAtoms():
                if atom2.getElement() in ['O', 'N']:  # Potential acceptors/donors
                    dist = pr.calcDistance(atom1, atom2)
                    if dist <= max_distance:  # Typical H-bond distance
                        hbonds.append((atom1.getCoords(), atom2.getCoords()))  # Store atom coordinates

    return hbonds

def Receptor3DView(receptorPDB, ligPDB):
    view = py3Dmol.view()
    view.setBackgroundColor('white')

    # Load the receptor
    view.addModel(open(receptorPDB, 'r').read(), 'pdb')

    # ❌ Explicitly REMOVE wire/cartoon representation
    view.addStyle({}, {'cartoon': {'color': 'white', 'opacity': 0}})

    # ✅ Add molecular surface for the receptor
    view.addSurface(py3Dmol.SAS, {'opacity': 0.7, 'color': 'red'})

    # Identify and highlight hydrogen-bonding residues
    hbond_residues = find_hbond_residues(receptorPDB, ligPDB)
    for resnum in hbond_residues:
        view.addStyle({'resi': str(resnum)}, {'stick': {}})  # Show only the hydrogen-bonding residues in sticks

    # Load the ligand
    if ligPDB is not None:
        view.addModel(open(ligPDB, 'r').read(), 'pdb')
        view.addStyle({'hetflag': True}, {'stick': {'colorscheme': 'greenCarbon'}})

    # Detect and add hydrogen bonds (as dashed yellow cylinders)
    hbonds = find_hydrogen_bonds(receptorPDB, ligPDB)
    for donor_coords, acceptor_coords in hbonds:
        view.addCylinder({
            'start': {'x': donor_coords[0], 'y': donor_coords[1], 'z': donor_coords[2]},
            'end': {'x': acceptor_coords[0], 'y': acceptor_coords[1], 'z': acceptor_coords[2]},
            'color': 'yellow',
            'radius': 0.1,
            'dashed': True
        })

    view.zoomTo()
    return view.show()

# Run visualization
Receptor3DView(receptorPDB=receptorPDB, ligPDB=prody_ligandPDB)


In [ ]:
import py3Dmol
import prody as pr

def find_hbond_residues(receptorPDB, ligPDB, max_distance=1):
    """Identify receptor residues involved in hydrogen bonding with the ligand."""
    receptor = pr.parsePDB(receptorPDB)
    ligand = pr.parsePDB(ligPDB)

    hbond_residues = set()

    for atom1 in receptor.iterAtoms():
        if atom1.getElement() in ['O', 'N']:  # Potential acceptors/donors
            for atom2 in ligand.iterAtoms():
                if atom2.getElement() in ['O', 'N']:  # Potential acceptors/donors
                    dist = pr.calcDistance(atom1, atom2)
                    if dist <= max_distance:  # Typical H-bond distance
                        hbond_residues.add(atom1.getResnum())  # Store residue number

    return hbond_residues

def find_hydrogen_bonds(receptorPDB, ligPDB, max_distance=3.5):
    """Find hydrogen bonds based on atom distances and return coordinate pairs."""
    receptor = pr.parsePDB(receptorPDB)
    ligand = pr.parsePDB(ligPDB)

    hbonds = []

    for atom1 in receptor.iterAtoms():
        if atom1.getElement() in ['O', 'N']:  # Potential acceptors/donors
            for atom2 in ligand.iterAtoms():
                if atom2.getElement() in ['O', 'N']:  # Potential acceptors/donors
                    dist = pr.calcDistance(atom1, atom2)
                    if dist <= max_distance:  # Typical H-bond distance
                        hbonds.append((atom1.getCoords(), atom2.getCoords()))  # Store atom coordinates

    return hbonds

def Receptor3DView(receptorSDF, ligPDB):
    view = py3Dmol.view()
    view.setBackgroundColor('white')

    # Load the receptor **from the SDF file**
    view.addModel(open(receptorSDF, 'r').read(), 'pdb')

    # ✅ Cartoon representation for **ENTIRE SDF**
    view.addStyle({}, {'cartoon': {'color': 'spectrum', 'opacity': 0.8}})

    # ✅ Add molecular surface for extra context
    #view.addSurface(py3Dmol.SAS, {'opacity': 0.4, 'color': 'gray'})

    # Identify and highlight hydrogen-bonding residues
    hbond_residues = find_hbond_residues(receptorSDF, ligPDB)
    for resnum in hbond_residues:
        view.addStyle({'resi': str(resnum)}, {'stick': {}})  # Show only the hydrogen-bonding residues in sticks

    # Load the ligand
    if ligPDB is not None:
        view.addModel(open(ligPDB, 'r').read(), 'pdb')
        view.addStyle({'hetflag': True}, {'stick': {'colorscheme': 'greenCarbon'}})

    # Detect and add hydrogen bonds (as dashed yellow cylinders)
    hbonds = find_hydrogen_bonds(receptorSDF, ligPDB)
    for donor_coords, acceptor_coords in hbonds:
        view.addCylinder({
            'start': {'x': donor_coords[0], 'y': donor_coords[1], 'z': donor_coords[2]},
            'end': {'x': acceptor_coords[0], 'y': acceptor_coords[1], 'z': acceptor_coords[2]},
            'color': 'yellow',
            'radius': 0.1,
            'dashed': True
        })

    view.zoomTo()
    return view.show()

# Run visualization using SDF for the receptor
Receptor3DView(receptorSDF=receptorPDB, ligPDB=prody_ligandPDB)


In [ ]:
!obabel {outputPDBQT} -O output.sdf

In [ ]:
'''
def Receptor3DView(receptorPDB = None, boxPDB = None, ligPDB = None):

    view = py3Dmol.view()
    view.setBackgroundColor('white')

    #view.addModel(open(boxPDB, 'r').read(),'pdb')
    #view.addStyle({'stick': {}})
    #view.zoomTo()

    view.addModel(open(receptorPDB, 'r').read(),'pdb')
    view.addStyle({'cartoon': {'color':'spectrum', 'opacity': 0.5}})

    if ligPDB is not None:
      view.addModel(open(ligPDB, 'r').read(), 'pdb')
      view.addStyle({'hetflag': True}, {'stick': {}})

    hbonds = find_hydrogen_bonds(receptorPDB, ligPDB)

    # Add hydrogen bonds to visualization
    for donor, acceptor in hbonds:
        view.addCylinder({
            'start': {'x': donor[0], 'y': donor[1], 'z': donor[2]},
            'end': {'x': acceptor[0], 'y': acceptor[1], 'z': acceptor[2]},
            'color': 'yellow',
            'radius': 0.1,
            'dashed': True
        })

    return view

def Complex3DView(view, ligmol = None, refligPDB = None, reflig_resn = None):

    new_viewer = copy.deepcopy(view)

    mblock = Chem.MolToMolBlock(ligmol)
    new_viewer.addModel(mblock, 'mol')
    new_viewer.addStyle({'hetflag': True}, {"stick": {'colorscheme': 'greenCarbon'}})

    if refligPDB is not None and reflig_resn is not None:
      new_viewer.addModel(open(refligPDB, 'r').read(), 'pdb')
      new_viewer.addStyle({'resn': reflig_resn}, {"stick": {'colorscheme': 'magentaCarbon', 'opacity': 0.8}})

    return new_viewer

dock_outSDF = 'output.sdf'
confs = Chem.SDMolSupplier(dock_outSDF)

def conf_viewer(idx):
    mol = confs[idx]
    return Complex3DView(Receptor3DView(receptorPDB = receptorPDB, boxPDB = None, ligPDB = prody_ligandPDB), \
                         mol, \
                         refligPDB = None, reflig_resn = None).show()


interact(conf_viewer, idx=ipywidgets.IntSlider(min=0, max=len(confs)-1, step=1))
'''

In [ ]:
import prody as pr
import py3Dmol

# Paths to files
new_protein = receptorPDB  # new protein (design)
old_protein = orig_protein  # old protein (to align)
ligand_protein = prody_ligandPDB  # ligand

# Important residues
new_residues = new_aligns
old_residues = orig_aligns

#Convert residue lists to space-separated strings for selection
new_residues_str = ' '.join(str(res) for res in new_residues)
old_residues_str = ' '.join(str(res) for res in old_residues)

# Select corresponding CA atoms for alignment using the variables
new_sel = new_structure.select(f'name CA and resid {new_residues_str}')
old_sel = old_structure.select(f'name CA and resid {old_residues_str}')

# Check that both selections are valid and match in size
if new_sel is None or old_sel is None:
    raise ValueError('One of the selections returned None. Check residue numbers!')
if new_sel.numAtoms() != old_sel.numAtoms():
    raise ValueError('Selections have different numbers of atoms!')

# Perform the alignment
transformation = pr.calcTransformation(old_sel, new_sel)
transformation.apply(old_structure)

# Save aligned old protein
aligned_old_protein = 'aligned_old_protein.pdb'
pr.writePDB(aligned_old_protein, old_structure)

# ---- Step 2: Visualize ---- #
def CompareProteins3DView(newPDB, oldPDB, ligPDB=None):
    view = py3Dmol.view()
    view.setBackgroundColor('white')

    # Add NEW protein
    new_data = open(newPDB, 'r').read()
    view.addModel(new_data, 'pdb')
    view.setStyle({'model': 0}, {'cartoon': {'color': 'spectrum', 'opacity': 0.5}})

    # Highlight important NEW residues
    for resi in new_residues:
        resi_str = str(resi)
        view.addStyle({'model': 0, 'resi': resi_str}, {'stick': {'colorscheme': 'magentaCarbon'}})
        view.addLabel(f"New Res {resi_str}", {
            'fontColor': 'black',
            'backgroundColor': 'white',
            'showBackground': True,
            'backgroundOpacity': 0.7,
            'fontSize': 12
        }, {'model': 0, 'resi': resi_str})

    # Add aligned OLD protein
    old_data = open(oldPDB, 'r').read()
    view.addModel(old_data, 'pdb')
    view.setStyle({'model': 1}, {'cartoon': {'color': 'lightblue', 'opacity': 0.5}})

    # Highlight important OLD residues
    for resi in old_residues:
        resi_str = str(resi)
        view.addStyle({'model': 1, 'resi': resi_str}, {'stick': {'colorscheme': 'blueCarbon'}})
        view.addLabel(f"Old Res {resi_str}", {
            'fontColor': 'black',
            'backgroundColor': 'white',
            'showBackground': True,
            'backgroundOpacity': 0.7,
            'fontSize': 12
        }, {'model': 1, 'resi': resi_str})

    # Add ligand (optional)
    if ligPDB is not None:
        ligand_data = open(ligPDB, 'r').read()
        view.addModel(ligand_data, 'pdb')
        view.setStyle({'model': 2, 'hetflag': True}, {'stick': {'colorscheme': 'greenCarbon'}})

    view.zoomTo()
    return view

# Run the visualization with aligned old protein
CompareProteins3DView(
    newPDB=new_protein,
    oldPDB=aligned_old_protein,
    ligPDB=ligand_protein
).show()


In [ ]:
import pymol
from pymol import cmd

# Start PyMOL (if not already running)
#pymol.finish_launching()

# Load both structures
cmd.load(f'{new_protein}', 'new_protein')
cmd.load(f'{orig_protein}', 'old_protein')

# Align old protein onto new protein using the important residues
new_residues = '+'.join(str(res) for res in new_aligns)
old_residues = '+'.join(str(res) for res in orig_aligns)

# Align using specific residues
cmd.align(f'old_protein and name CA and resi {old_residues}',
          f'new_protein and name CA and resi {new_residues}')

# Optional: color them for clarity
cmd.color('cyan', 'old_protein')
cmd.color('magenta', 'new_protein')

# Show cartoons
cmd.show('cartoon', 'old_protein')
cmd.show('cartoon', 'new_protein')

# Optional: save the aligned old protein
cmd.save('aligned_new_protein.pdb', 'new_protein')
cmd.save('aligned_old_protein.pdb', 'old_protein')


In [ ]:
import py3Dmol

def show_aligned_proteins(new_pdb, old_pdb):
    view = py3Dmol.view()
    view.setBackgroundColor('white')

    # Add aligned new protein
    with open(new_pdb, 'r') as f:
        new_data = f.read()
    view.addModel(new_data, 'pdb')
    view.setStyle({'model': 0}, {'cartoon': {'color': 'magenta', 'opacity': 0.7}})

    # Add aligned old protein
    with open(old_pdb, 'r') as f:
        old_data = f.read()
    view.addModel(old_data, 'pdb')
    view.setStyle({'model': 1}, {'cartoon': {'color': 'cyan', 'opacity': 0.7}})

    view.zoomTo()
    return view

# Run the visualization
show_aligned_proteins('aligned_new_protein.pdb', 'aligned_old_protein.pdb').show()
